In [10]:
import re
import os
from pathlib import Path
from typing import Union
from warnings import simplefilter
from datetime import date

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [87]:
test = Path(r'C:\Users\wonhyeong\downloads\20211020_10-Q_edgar_data_1456802_0001477932-21-007463.txt') #ixbrl
# test = Path(r'C:\Users\wonhyeong\downloads\20211001_10-K_edgar_data_1338929_0001477932-21-006965.txt') #html with unclosed body/html tag

# with open(test, 'r', encoding='utf-8') as f:
#     soup = BeautifulSoup(f, 'html.parser')

# print(soup)

In [ ]:
import logging
from xbrl.cache import HttpCache
from xbrl.instance import XbrlParser, XbrlInstance
# just to see which files are downloaded
logging.basicConfig(level=logging.INFO)

cache: HttpCache = HttpCache('./cache')
cache.set_headers({'From': 'YOUR@EMAIL.com', 'User-Agent': 'py-xbrl/2.1.0'})
parser = XbrlParser(cache)

schema_url = "https://www.sec.gov/Archives/edgar/data/0000320193/000032019321000105/aapl-20210925.htm"
inst: XbrlInstance = parser.parse_instance(schema_url)

In [113]:
# print json to console
inst.json(file_path='test.json', override_fact_ids=True)


In [158]:
from lxml.html.clean import Cleaner, fromstring
import lxml.html


In [198]:
root = Path(r'C:\Users\wonhyeong\workings\data\10X\cleaned')
root_dir = [x for x in root.iterdir() if x.is_dir()]
# list of files
sub_dir = [list(f.iterdir()) for f in root_dir if f.is_dir()]
print(sub_dir)

[[WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1993/QTR1'), WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1993/QTR2'), WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1993/QTR3'), WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1993/QTR4')], [WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1994/QTR1'), WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1994/QTR2'), WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1994/QTR3'), WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1994/QTR4')], [WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1995/QTR1'), WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1995/QTR2'), WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1995/QTR3'), WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1995/QTR4')], [WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1996/QTR1'), WindowsPath('C:/Users/wonhyeong/workings/data/10X/cleaned/1996/QTR2')

In [205]:
def clean_html(file, done, errored: Union[str, Path]) -> str:
    def hidden(tag: BeautifulSoup) -> bool:
        text = re.compile('display\s*?\:\s*?none')
        style = tag.get('style', '')
        return bool(text.search(style))
    try:
        with open(file, encoding='utf-8') as f:
            b = f.read()
            soup = BeautifulSoup(b, 'html.parser')
            header = soup.find('sec-header')
            body = soup.find('html')
            for tag in body.find_all():
                tag: BeautifulSoup
                if hidden(tag) or not tag.contents:
                    tag.replaceWith('')
                #clear attributes
                for attr in ["class", "id", "name", "style"]:
                    del tag[attr]
            f.close()
    except:
        try:
            f.close()
        except:
            pass
        save_on_failure(file, errored)
        return False
    save_on_success(file, '\n'.join([str(header), str(body)]), done)
    return True

def save_on_success(file: Union[str, Path], text: str, done) -> None:
    with open(file, 'w', encoding='utf-8') as f:
        f.write(text)
        f.close()
    file.rename(done / file.name)
    

def save_on_failure(file: Path, errored_dir: Path) -> None:
    errored_dir.mkdir(exist_ok=True)
    file.rename(errored_dir / file.name)


In [ ]:
for years in root_dir:
    for quarters in sub_dir:
        for qtr in quarters:
            name = '_'.join([str(qtr.name),'cleaned'])
            cleaned = qtr.parent / name
            done= cleaned / 'done'
            errored = cleaned / 'errored'
            cleaned.mkdir(exist_ok=True)
            done.mkdir(exist_ok=True)
            errored.mkdir(exist_ok=True)
            log = ''
            for file in files:
                ok = clean_html(file, done, errored)
                '\n'.join([log, f"{file.name} : {ok}"])
                # save log file
            with open(cleaned / 'log.txt', 'w', encoding='utf-8') as f: 
                f.write(log)
                f.close()

C:\Users\wonhyeong\workings\data\10X\cleaned\1993\QTR1 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1993\QTR2 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1993\QTR3 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1993\QTR4 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1994\QTR1 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1994\QTR2 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1994\QTR3 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1994\QTR4 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1995\QTR1 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1995\QTR2 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1995\QTR3 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1995\QTR4 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1996\QTR1 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1996\QTR2 is being 

In [209]:
qtr = Path(r'C:\Users\wonhyeong\workings\data\10X\QTR4')

name = '_'.join([str(qtr.name),'cleaned'])
cleaned = qtr.parent / name
done= cleaned / 'done'
errored = cleaned / 'errored'
cleaned.mkdir(exist_ok=True)
done.mkdir(exist_ok=True)
errored.mkdir(exist_ok=True)
files = list(qtr.iterdir())
log = ''
for file in files:
    ok = clean_html(file, done, errored)
    '\n'.join([log, f"{file.name} : {ok}"])
    # save log file
with open(cleaned / 'log.txt', 'w', encoding='utf-8') as f: 
    f.write(log)
    f.close()


In [201]:
for years in root_dir:
    for quarters in sub_dir:
        for qtr in quarters:
            name = '_'.join([str(qtr.name),'cleaned'])
            cleaned = qtr.parent / name
            done= cleaned / 'done'
            errored = cleaned / 'errored'
            cleaned.mkdir(exist_ok=True)
            done.mkdir(exist_ok=True)
            errored.mkdir(exist_ok=True)
            for file in qtr:
                done = clean_html(file, done, errored)
        break

C:\Users\wonhyeong\workings\data\10X\cleaned\1993\QTR1 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1993\QTR2 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1993\QTR3 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1993\QTR4 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1994\QTR1 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1994\QTR2 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1994\QTR3 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1994\QTR4 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1995\QTR1 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1995\QTR2 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1995\QTR3 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1995\QTR4 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1996\QTR1 is being cleaned
C:\Users\wonhyeong\workings\data\10X\cleaned\1996\QTR2 is being 

In [ ]:
dir = Path(r'C:\Users\wonhyeong\workings\data\10X\QTR4')
# list of files
files = [dir / f for f in os.listdir(dir)]
def clean_html(file: Union[str, Path]) -> pd.DataFrame:
    """Clean HTML file and return a dataframe"""
    style = re.compile('\s*?style\=\".*?\".*?', flags=re.I)
    with open(file, encoding='utf-8') as f:
        soup = BeautifulSoup(f, 'html.parser')
    soup = soup.find('body')
    # remove all script and style elements
    for tag in soup():
        print(tag())
        break
        # for attribute in ["class", "id", "name", "style"]:
        #     del tag[attribute]
    return str(soup)

b = clean_html(files[0])
# print(b)